In [ ]:
import numpy as np
import pandas as pd
import tushare as ts
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from scipy import stats
from datetime import datetime, timedelta
from qvix import get_qvix, update_qvix, get_hist_vol
from option_backtest import FixedHedge, DynamicHedge


def get_iv_hv_data(underlying: str):
    qvix_df = get_qvix(underlying)
    hist_vol = get_hist_vol(underlying)
    hist_vol['HV_20_40'] = (hist_vol['HV_20'] + hist_vol['HV_40']) / 2
    hist_vol['date'] = hist_vol['date'].map(lambda d_str: datetime.strptime(d_str, '%Y-%m-%d'))
    hist_vol.set_index('date', inplace=True)
    return qvix_df, hist_vol


def plot_iv_hv(qvix_df: pd.DataFrame, hv_df: pd.DataFrame):
    disp_list = [qvix_df['close'], hv_df['HV_20_40']]
    df_combine = pd.concat(disp_list, axis=1).dropna()
    df_combine.head()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_combine.index, y=df_combine['HV_20_40'], mode='lines+markers', name='HV'))
    fig.add_trace(go.Scatter(x=df_combine.index, y=df_combine['close'], mode='lines+markers', name='IV'))
    fig.show()
    
    
def plot_box(qvix_df: pd.DataFrame, hv_df: pd.DataFrame):
    fig = go.Figure()
    fig.add_trace(go.Box(y=qvix_df['close'], name='qvix', boxpoints='all'))
    fig.add_trace(go.Box(y=hv_df['HV_20_40'], name='hv_20&40', boxpoints='all'))
    # fig.update_traces(boxpoints='all', jitter=0)
    fig.show()
    

def plot_hv_cone(underlying: str):
    hist_vol = get_hist_vol(underlying)
    columns = [name for name in hist_vol.columns if name.startswith('HV')]
    quantile_df = hist_vol[columns].quantile(np.arange(.1, 1.1, .1))
    quantile_df.index = list(range(10, 110, 10))

    fig = go.Figure()
    for index, name in quantile_df.iterrows():
        fig.add_trace(go.Scatter(x=columns, y=quantile_df.loc[index].values, mode='lines+markers', name=str(index)))

    fig.show()

In [ ]:
# 从期权论坛更新qvix数据
_ = update_qvix('510050')
_ = update_qvix('510300')

In [ ]:
# 获取510050和510300数据
qvix_510050, hist_vol_510050 = get_iv_hv_data('510050')
qvix_510300, hist_vol_510300 = get_iv_hv_data('510300')
# qvix_510050.head()
# hist_vol_510050.head()
# qvix_510300.head()
# hist_vol_510300.head()

In [ ]:
# 绘制510050 IV和HV走势图
plot_iv_hv(qvix_510050, hist_vol_510050)

In [ ]:
# 绘制510300 IV和HV走势图
plot_iv_hv(qvix_510300, hist_vol_510300)

In [ ]:
# 绘制510050 qvix和hv盒须图以及散点分布
plot_box(qvix_510050, hist_vol_510050)

In [ ]:
# 绘制510300 qvix和hv盒须图以及散点分布
plot_box(qvix_510300, hist_vol_510300)

In [ ]:
# 绘制510050历史波动率锥
plot_hv_cone('510050')

In [ ]:
# 绘制510300历史波动率锥
plot_hv_cone('510300')

In [ ]:
# 绘制期权论坛qvix盒须图
fig = px.box(qvix_df, y='high', points='all')
fig.show()

In [ ]:
# IV20&40盒须图
fig = px.box(hist_vol, y='HV_20_40', points='all')
fig.show()

In [ ]:
# 绘制历史波动率锥
hist_vol = get_hist_vol()
columns = [name for name in hist_vol.columns if name.startswith('HV')]
quantile_df = hist_vol[columns].quantile(np.arange(.1, 1.1, .1))
quantile_df.index = list(range(10, 110, 10))

fig = go.Figure()
for index, name in quantile_df.iterrows():
    fig.add_trace(go.Scatter(x=columns, y=quantile_df.loc[index].values, mode='lines+markers', name=str(index)))

fig.show()

In [ ]:
# 统计510050qvix节假日前一个交易日的隐波涨跌情况
df = qvix_df
df['date'] = df.index
df['last_date'] = df['date'].shift(1)
df['last_close'] = df['close'].shift(1)
df.dropna(inplace=True)
df['vix_change'] = df['close'] - df['last_close']
df['interval'] = df['date'] - df['last_date']
df['id'] = np.arange(len(df))

# 筛选数据
holiday = df[df['interval'] > timedelta(days=3)]

# 筛选节假日前一天
pre_holiday_id_list = (holiday['id'] - 1).to_list()
pre_holiday_id_list
pre_holiday = df[df['id'].isin(pre_holiday_id_list)]

print('up count:', len(pre_holiday[pre_holiday.vix_change > 0]))
print('down count:', len(pre_holiday[pre_holiday.vix_change < 0]))
print('zero count:', len(pre_holiday[pre_holiday.vix_change == 0]))
print('all count:', len(pre_holiday))